<a href="https://colab.research.google.com/github/elliemci/chatbots/blob/main/summarization_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrated summarization NLP task into a telegram chatbot.

The user is able to choose a subject, the information about it is taken from Wikipedia and summarized using a model from Hugging Face library.

In [1]:
# install Python transformers(NLP) pyTelegramBotAPI(chatbot) and Wikipedia(data) libraries
!pip install transformers pyTelegramBotAPI wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.14.0-py3-none-any.whl size=215250 sha256=dab3405a9adfe00c9ab9517511da583431a0ca9fc1212c2dc6ce8da0e1e7ad17
  Stored in directory: /root/.cache/pip/wheels/25/51/2d/24b40a366c85c37928d5aa36ddf257e5a79fad25e1ecd11b2c
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=90045ce06c1fb509ca117bc7657ef436fcca2f02b1fbde9ab8f690bb41b253c2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built pyTelegramBotAPI wikipedia


In [5]:
import telebot
import wikipedia

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

print(wikipedia.summary("Wikipedia"))

The Wikimedia Foundation, Inc. (WMF) is an American 501(c)(3) nonprofit organization headquartered in San Francisco, California, and registered as a charitable foundation under local laws. It is best known as the host platform for Wikipedia, the largest crowdsourced online encyclopedia in the world. The organization also hosts other related projects and MediaWiki, a wiki software.The Wikimedia Foundation was established in 2003 in St. Petersburg, Florida, by Jimmy Wales as a nonprofit way to fund Wikipedia, Wiktionary, and other crowdsourced wiki projects that had until then been hosted by Bomis, Wales's for-profit company. The Foundation finances itself mainly through millions of small donations from Wikipedia readers, collected through email campaigns and annual fundraising banners placed on Wikipedia and its sister projects. These are complemented by grants from philanthropic organizations and tech companies, and starting in 2022, by services income from Wikimedia Enterprise.
The Fo

In [8]:
import os
from getpass import getpass

os.environ['TELEGRAM_BOT_TOKEN'] = getpass('Enter your bot token: ')
TOKEN = os.getenv('TELEGRAM_BOT_TOKEN')

# create the bot an instance of TeleBot setting the token
bot = telebot.TeleBot(TOKEN)

Enter your bot token: ··········


In [10]:
# load T5 transformer model from HuggingFace model hub
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")


# instantiate the Hugging Face papiline class by passing the "summarization" task
summarization_pipeline = pipeline("summarization", model="t5-base")

# initalize bot state and subject
state = 0
subject = ""

@bot.message_handler(func=lambda message: True)
def message(message):

    global state, subject
    chat_id = message.chat.id

    if state == 0:
      bot.send_message(chat_id, "Hi there! I'm ready get down to summarizing.")
      state = 1
    elif state == 1:
        subject = message.text
        if subject == "no":
          bot.send_message(chat_id, "Goodbye!")
          state = 0
        else:
          bot.send_message(chat_id, "Tell me a subject you want to summarize?")
          state = 2
    elif state == 2:
        subject = message.text
        try:
            wiki_text = wikipedia.summary(subject)
        except wikipedia.exceptions.DisambiguationError as e:
            bot.send_message(chat_id, "Your topic is ambiguous, please provide more specific subject.")
            return
        except wikipedia.exceptions.PageError as e:
            bot.send_message(chat_id, "I couldn't find a Wikipedia page on that subject.")
            return
        summary = summarization_pipeline(wiki_text, min_length=10, max_length=50, do_sample=False)[0]['summary_text']
        bot.send_message(chat_id, summary)
        state = 3
    elif state == 3:
      bot.send_message(chat_id, "You want me to keep summzrizing?")
      state = 1

bot.polling()